In [14]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np



In [15]:
    TRAIN_DIR='images/train'
    TEST_DIR='images/validation'

In [16]:
def createdataframe(dir):
    image_paths=[]
    labels=[]
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"completed")
    return image_paths,labels

In [17]:
train=pd.DataFrame()
train['image'],train['label']=createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [18]:
print(train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [19]:
test=pd.DataFrame()
test['image'],test['label']=createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [20]:
print(test)

                                    image     label
0       images/validation\angry\10052.jpg     angry
1       images/validation\angry\10065.jpg     angry
2       images/validation\angry\10079.jpg     angry
3       images/validation\angry\10095.jpg     angry
4       images/validation\angry\10121.jpg     angry
...                                   ...       ...
7061  images/validation\surprise\9806.jpg  surprise
7062  images/validation\surprise\9830.jpg  surprise
7063  images/validation\surprise\9853.jpg  surprise
7064  images/validation\surprise\9878.jpg  surprise
7065   images/validation\surprise\993.jpg  surprise

[7066 rows x 2 columns]


In [21]:
from tqdm.notebook import tqdm

In [22]:
def extract_features(images):
    features=[]
    for image in tqdm(images):
        img=load_img(image,grayscale = True)
        img=np.array(img)
        features.append(img)
    features=np.array(features)
    features=features.reshape(len(features),48,48,1)
    return features


In [23]:
train_features=extract_features(train['image'])                        
                                

  0%|          | 0/28821 [00:00<?, ?it/s]

G:\DLCA\emoji detection system\lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [24]:
test_features=extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [ ]:
x_train=train_features/255.0
x_test=test_features/255.0

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()
le.fit(train['label'])

In [ ]:
y_train=le.transform(train['label'])
y_test=le.transform(test['label'])

In [ ]:
y_train=to_categorical(y_train,num_classes=7)
y_test=to_categorical(y_test,num_classes=7)

In [ ]:
model=Sequential()
# convolutional layers

model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

#output layer
model.add(Dense(7, activation='softmax'))







In [ ]:
model.compile(optimizer='adam',loss= 'categorical_crossentropy', metrics =['accuracy'])

In [ ]:
model.fit(x=x_train,y=y_train, batch_size=128, epochs=100 ,validation_data=(x_test,y_test))

In [ ]:
model_json=model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [ ]:
from keras.models import model_from_json

In [ ]:
json_file=open("emotiondetector.json", "r")
model_json=json_file.read()
json_file.close()
model=model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [ ]:
label = ['angry', 'disgust', 'fear' , 'happy' ,'neutral', 'sad' , 'surprise']

In [ ]:
def ef(image):
    img=load_img(image,grayscale = True)
    feature=np.array(img)
    feature=feature.reshape(1,48,48,1)
    return feature/255.0

In [ ]:
image= 'images/train/sad/6.jpg'
print("orignal image is of angry")
img=ef(image)
pred=model.predict(img)
pred_label=label[pred.argmax()]
print("the model's prediction is ",pred_label)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
image= 'images/train/sad/42.jpg'
print("orignal image is of sad")
image=ef(image)
pred=model.predict(img)
pred_label=label[pred.argmax()]
print("model prediction is ", pred_label)
plt.imshow(img.reshape(48,48), cmap='grey')

In [ ]:
image= 'images/train/angry/27.jpg'
print("orignal image is of sad")
image=ef(image)
pred=model.predict(img)
pred_label=label[pred.argmax()]
print("model prediction is ", pred_label)
plt.imshow(img.reshape(48,48), cmap='grey')